# Análise de Jogos da NFL com Regressão Logística

Este notebook utiliza a regressão logística para prever o vencedor de jogos da NFL com base em vitórias e derrotas acumuladas dos times. A seguir, apresentamos o passo a passo do código:

---

## 1. Importação de Bibliotecas

Importamos as bibliotecas necessárias para manipulação de dados, divisão de conjuntos, criação do modelo e avaliação de desempenho.

## 2. Carregamento do dataset

Utilizei o datset "games.csv", que contém dados de jogos da NFL de 1999 até os jogos atuais, o dataset é atualizado toda semana e eu atualizo através desse repositório: "https://github.com/nflverse/nfldata/blob/master/data/games.csv"

## 3. Filtragem dos dados
Filtrei apenas os jogos de temporada regular, de 2013 pra frente

## 4. Criação de features

Vitória e derrotas acumuladas no time de casa e fora

## 5. Divisão em treino e teste

Utilizei 80% dos dados para treino e 20% para teste

## 6. Treinamento do modelo

Criei um moelo de regressão logística, que visa predizer o vencedor da partida

## 7. Avaliação do modelo

Calcula a acurácia do modelo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Carregar dataset
df = pd.read_csv('games.csv')

# Filtrar temporadas a partir de 2013 e jogos da temporada regular
df = df[df['season'] >= 2013]
df = df[df['game_type'] == 'REG']

# Definir o vencedor (1 se o time da casa vencer, 0 se o time visitante vencer)
df['winner'] = df['result'].apply(lambda x: 1 if x > 0 else 0)
df = df[df['result'] != 0]

# Contar vitórias do time da casa
df['home_team_wins'] = df.groupby(['home_team', 'season'])['winner'].cumsum()

# Contar vitórias do time visitante (inverso das vitórias do time da casa)
df['away_team_wins'] = df.groupby(['away_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time da casa (inverso das vitórias do time da casa)
df['home_team_losses'] = df.groupby(['home_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time visitante
df['away_team_losses'] = df.groupby(['away_team', 'season'])['winner'].cumsum()

# Selecionar as colunas de interesse para o modelo
parametros = ['home_team_wins', 'away_team_wins', 'home_team_losses', 'away_team_losses']

# Dividir em treino e teste
df_train = df[df['season'] < 2023]
df_test = df[df['season'] >= 2023]

X_train = df_train[parametros]
y_train = df_train['winner']
X_test = df_test[parametros]
y_test = df_test['winner']

# Criar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100:.2f}%')


Acurácia: 84.19%


In [2]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(modelo, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'Acurácia média com cross-validation: {scores.mean() * 100:.2f}%')


Acurácia média com cross-validation: 84.38%


In [3]:
# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Fazer previsões
y_pred_semana = modelo.predict(df_semana[parametros])

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = y_pred_semana

# Função para exibir a predição com os nomes dos times
def display_prediction(row):
    if row['predicted_winner'] == 1:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['home_team']}"
    else:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['away_team']}"

# Aplicar a função no DataFrame de predições
df_semana['prediction_display'] = df_semana.apply(display_prediction, axis=1)

# Exibir as predições para todos os jogos da semana escolhida
for pred in df_semana['prediction_display']:
    print(pred)


Digite o número da semana (por exemplo, 1 para semana 1):  13


DET vs CHI, vencedor: DET
DAL vs NYG, vencedor: NYG
GB vs MIA, vencedor: GB
KC vs LV, vencedor: KC
ATL vs LAC, vencedor: LAC
CIN vs PIT, vencedor: PIT
JAX vs HOU, vencedor: HOU
MIN vs ARI, vencedor: MIN
NE vs IND, vencedor: IND
NYJ vs SEA, vencedor: SEA
WAS vs TEN, vencedor: WAS
CAR vs TB, vencedor: TB
NO vs LA, vencedor: LA
BAL vs PHI, vencedor: PHI
BUF vs SF, vencedor: BUF
DEN vs CLE, vencedor: DEN


In [4]:
def calcular_lucro(row):
    aposta = 100  # Valor fixo da aposta em cada jogo
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta
    else:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta

In [5]:
def moneyline_to_decimal(moneyline):
    if moneyline > 0:
        return (moneyline / 100) + 1
    elif moneyline < 0:
        return (100 / abs(moneyline)) + 1
    else:
        return None  # Odds inválidas

In [6]:
# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Converter odds Moneyline para Decimais
df_semana['home_odds_decimal'] = df_semana['home_moneyline'].apply(moneyline_to_decimal)
df_semana['away_odds_decimal'] = df_semana['away_moneyline'].apply(moneyline_to_decimal)

# Fazer previsões
y_pred_semana = modelo.predict(df_semana[parametros])

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = y_pred_semana

# Função para exibir a predição com os nomes dos times e odds decimais
def display_prediction(row):
    predicted_team = row['home_team'] if row['predicted_winner'] == 1 else row['away_team']
    real_winner_team = (
        row['home_team'] if row['winner'] == 1 
        else row['away_team'] if row['winner'] == 0 
        else "Ainda não definido"
    )
    return (
        f"{row['home_team']} vs {row['away_team']}, "
        f"vencedor previsto: {predicted_team}, "
        f"vencedor real: {real_winner_team}, "
        f"odds decimais: casa {row['home_odds_decimal']:.2f}, fora {row['away_odds_decimal']:.2f}"
    )

# Aplicar a função no DataFrame de predições
df_semana['prediction_display'] = df_semana.apply(display_prediction, axis=1)

# Exibir as predições para todos os jogos da semana escolhida
print("\nPredições para os jogos da semana:")
for pred in df_semana['prediction_display']:
    print(pred)

# Calcular o lucro de cada jogo
df_semana['lucro'] = df_semana.apply(calcular_lucro, axis=1)

# Calcular métricas da semana
acertos = (df_semana['predicted_winner'] == df_semana['winner']).sum()
total_jogos = len(df_semana[df_semana['winner'].notna()])  # Ignorar jogos sem resultado
assertividade = (acertos / total_jogos * 100) if total_jogos > 0 else 0
lucro_total = df_semana['lucro'].sum()
porcentagem_lucro = (lucro_total / (total_jogos * 100)) * 100 if total_jogos > 0 else 0

# Exibir resultado geral da semana
print("\nResultados da Semana:")
print(f"Total de jogos: {total_jogos}")
print(f"Acertos: {acertos}")
print(f"Assertividade: {assertividade:.2f}%")
print(f"Lucro total: ${lucro_total:.2f}")
print(f"Porcentagem de lucro: {porcentagem_lucro:.2f}%")

Digite o número da semana (por exemplo, 1 para semana 1):  13



Predições para os jogos da semana:
DET vs CHI, vencedor previsto: DET, vencedor real: DET, odds decimais: casa 1.19, fora 4.90
DAL vs NYG, vencedor previsto: NYG, vencedor real: DAL, odds decimais: casa 1.46, fora 2.85
GB vs MIA, vencedor previsto: GB, vencedor real: GB, odds decimais: casa 1.57, fora 2.45
KC vs LV, vencedor previsto: KC, vencedor real: KC, odds decimais: casa 1.12, fora 6.50
ATL vs LAC, vencedor previsto: LAC, vencedor real: LAC, odds decimais: casa 1.98, fora 1.85
CIN vs PIT, vencedor previsto: PIT, vencedor real: PIT, odds decimais: casa 1.65, fora 2.30
JAX vs HOU, vencedor previsto: HOU, vencedor real: HOU, odds decimais: casa 2.60, fora 1.52
MIN vs ARI, vencedor previsto: MIN, vencedor real: MIN, odds decimais: casa 1.54, fora 2.54
NE vs IND, vencedor previsto: IND, vencedor real: IND, odds decimais: casa 2.14, fora 1.74
NYJ vs SEA, vencedor previsto: SEA, vencedor real: SEA, odds decimais: casa 1.87, fora 1.95
WAS vs TEN, vencedor previsto: WAS, vencedor real: W